In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/json-data/assignment_3_ai_tutors_dataset.json


In [25]:
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import json
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
class FocalLossWithWeights(nn.Module):
    def __init__(self, weight=None, gamma=2.0):
        super(FocalLossWithWeights, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none', weight=weight)

    def forward(self, inputs, targets):
        logp = self.ce(inputs, targets)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()


In [28]:
# ---------- Dataset Definition ----------
class TutorEvalSingleTaskDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

In [29]:
# ---------- Preprocessing ----------
def load_and_flatten(json_path):
    with open(json_path) as f:
        data = json.load(f)

    rows = []
    for instance in data:
        convo_id = instance["conversation_id"]
        history = instance["conversation_history"]
        for tutor_id, tutor_data in instance["tutor_responses"].items():
            row = {
                "conversation_id": convo_id,
                "tutor_id": tutor_id,
                "conversation_history": history,
                "tutor_response": tutor_data["response"],
                "Mistake_Identification": tutor_data["annotation"]["Mistake_Identification"],
                "Mistake_Location": tutor_data["annotation"]["Mistake_Location"],
                "Pedagogical_Guidance": tutor_data["annotation"]["Providing_Guidance"],
                "Actionability": tutor_data["annotation"]["Actionability"]
            }
            rows.append(row)
    return pd.DataFrame(rows)

In [30]:
def build_input_text(row):
    return f"Context:\n{row['conversation_history']}\n\nTutor Response:\n{row['tutor_response']}"

LABEL_MAP = {"Yes": 0, "To some extent": 1, "No": 2}
MERGED_LABEL_MAP = {"Yes": 1, "To some extent": 1, "No": 0}

def encode_labels(df):
    for task in ["Mistake_Identification", "Mistake_Location", "Pedagogical_Guidance", "Actionability"]:
        df[f"{task}_label"] = df[task].map(LABEL_MAP)
        df[f"{task}_binary"] = df[task].map(MERGED_LABEL_MAP)
    return df

def tokenize_inputs(tokenizer, texts, max_length=256):
    return tokenizer(
        texts,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )

In [31]:
def preprocess_dataset(json_path, task_label):
    df = load_and_flatten(json_path)
    df["input_text"] = df.apply(build_input_text, axis=1)
    df = encode_labels(df)

    train_df, val_df = train_test_split(df, test_size=0.1, stratify=df[task_label], random_state=42)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    train_enc = tokenize_inputs(tokenizer, train_df["input_text"].tolist())
    val_enc = tokenize_inputs(tokenizer, val_df["input_text"].tolist())

    train_labels = torch.tensor(train_df[task_label].tolist())
    val_labels = torch.tensor(val_df[task_label].tolist())

    train_dataset = TutorEvalSingleTaskDataset(train_enc, train_labels)
    val_dataset = TutorEvalSingleTaskDataset(val_enc, val_labels)

    return train_dataset, val_dataset, tokenizer, df


In [32]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def get_class_weights(labels, num_classes):
    class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=labels)
    return torch.tensor(class_weights, dtype=torch.float).to(device)


In [33]:
# ---------- Model ----------
class SingleTaskBertClassifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).logits

In [34]:
# ---------- Training ----------
def evaluate_model(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels)

    acc = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Validation Accuracy: {acc:.4f}  Validation Macro F1: {macro_f1:.4f}")
    print(classification_report(all_labels, all_preds, target_names=["Yes", "To some extent", "No"], zero_division=0))

def train_model(loss_type, train_loader, val_loader, num_labels, epochs=15):
    print(f"\n🔁 Training with: {loss_type.upper()} Loss\n")

    model = SingleTaskBertClassifier(num_labels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    train_labels_list = [label.item() for batch in train_loader for label in batch['labels']]
    class_weights = get_class_weights(train_labels_list, num_labels)

    if loss_type == "focal":
        criterion = FocalLossWithWeights(class_weights)
    elif loss_type == "smoothing":
        criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
    else:
        criterion = nn.CrossEntropyLoss(weight=class_weights)


    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")
        evaluate_model(model, val_loader)

        '''if(epoch==epochs):
            save_path = f"/kaggle/working/best_model_{loss_type}.pth"
            torch.save(model.state_dict(), save_path)
            print(f"✅ Model weights saved to {save_path}")'''



In [35]:
# ---------- Run ----------
json_path = "/kaggle/input/json-data/assignment_3_ai_tutors_dataset.json"
train_dataset, val_dataset, tokenizer, df = preprocess_dataset(json_path, "Mistake_Identification_label")
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)



In [36]:
df

,conversation_id,tutor_id,conversation_history,tutor_response,Mistake_Identification,Mistake_Location,Pedagogical_Guidance,Actionability,input_text,Mistake_Identification_label,Mistake_Identification_binary,Mistake_Location_label,Mistake_Location_binary,Pedagogical_Guidance_label,Pedagogical_Guidance_binary,Actionability_label,Actionability_binary
0,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Sonnet,"Tutor: Hi, could you please provide a step-by-...","Great, you've correctly identified the cost of...",Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1
1,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Llama318B,"Tutor: Hi, could you please provide a step-by-...",Now that we know the cost of 1 pound of meat i...,Yes,To some extent,To some extent,To some extent,"Context:\nTutor: Hi, could you please provide ...",0,1,1,1,1,1,1,1
2,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Llama31405B,"Tutor: Hi, could you please provide a step-by-...","You're close, but I notice that you calculated...",Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1
3,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,GPT4,"Tutor: Hi, could you please provide a step-by-...","That's correct. So, if 1 pound of meat costs $...",Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1
4,221-362eb11a-f190-42a6-b2a4-985fafdcfa9e,Mistral,"Tutor: Hi, could you please provide a step-by-...",It seems like you've calculated the cost as if...,Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,5910-25617a89-a4ae-47bb-8812-d6b39fa4e691,Mistral,"Tutor: Hi, could you please provide a step-by-...",It seems there might be a misunderstanding in ...,Yes,Yes,To some extent,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,1,1,0,1
2472,5910-25617a89-a4ae-47bb-8812-d6b39fa4e691,Phi3,"Tutor: Hi, could you please provide a step-by-...","To solve this problem, we need to add the numb...",No,No,No,No,"Context:\nTutor: Hi, could you please provide ...",2,0,2,0,2,0,2,0
2473,5910-25617a89-a4ae-47bb-8812-d6b39fa4e691,Sonnet,"Tutor: Hi, could you please provide a step-by-...",That's a great start and I like how you worked...,Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1
2474,5910-25617a89-a4ae-47bb-8812-d6b39fa4e691,Expert,"Tutor: Hi, could you please provide a step-by-...",Okay. So Hector gave 5 less than four times as...,Yes,Yes,Yes,Yes,"Context:\nTutor: Hi, could you please provide ...",0,1,0,1,0,1,0,1


In [37]:
for loss_name in ["ce", "smoothing", "focal"]:
    train_model(loss_name, train_loader, val_loader, num_labels=3)


🔁 Training with: CE Loss



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/15: 100%|██████████| 557/557 [01:58<00:00,  4.70it/s]


Epoch 1 Loss: 1.0134
Validation Accuracy: 0.7823  Validation Macro F1: 0.2926
                precision    recall  f1-score   support

           Yes       0.78      1.00      0.88       194
To some extent       0.00      0.00      0.00        17
            No       0.00      0.00      0.00        37

      accuracy                           0.78       248
     macro avg       0.26      0.33      0.29       248
  weighted avg       0.61      0.78      0.69       248



Epoch 2/15: 100%|██████████| 557/557 [01:57<00:00,  4.73it/s]


Epoch 2 Loss: 0.9611
Validation Accuracy: 0.7702  Validation Macro F1: 0.3642
                precision    recall  f1-score   support

           Yes       0.79      0.97      0.87       194
To some extent       0.30      0.18      0.22        17
            No       0.00      0.00      0.00        37

      accuracy                           0.77       248
     macro avg       0.36      0.38      0.36       248
  weighted avg       0.64      0.77      0.70       248



...

Epoch 13 Loss: 0.3344
Validation Accuracy: 0.3468  Validation Macro F1: 0.3035
                precision    recall  f1-score   support

           Yes       0.95      0.29      0.44       194
To some extent       0.10      0.47      0.16        17
            No       0.21      0.59      0.31        37

      accuracy                           0.35       248
     macro avg       0.42      0.45      0.30       248
  weighted avg       0.78      0.35      0.40       248



Epoch 14/15: 100%|██████████| 557/557 [01:57<00:00,  4.73it/s]


Epoch 14 Loss: 0.3509
Validation Accuracy: 0.3589  Validation Macro F1: 0.3080
                precision    recall  f1-score   support

           Yes       0.92      0.31      0.47       194
To some extent       0.10      0.47      0.16        17
            No       0.20      0.54      0.30        37

      accuracy                           0.36       248
     macro avg       0.41      0.44      0.31       248
  weighted avg       0.76      0.36      0.42       248



Epoch 15/15: 100%|██████████| 557/557 [01:57<00:00,  4.73it/s]


Epoch 15 Loss: 0.3292
Validation Accuracy: 0.3710  Validation Macro F1: 0.3140
                precision    recall  f1-score   support

           Yes       0.93      0.32      0.48       194
To some extent       0.10      0.41      0.16        17
            No       0.21      0.59      0.31        37

      accuracy                           0.37       248
     macro avg       0.41      0.44      0.31       248
  weighted avg       0.76      0.37      0.43       248

